<a href="https://colab.research.google.com/github/ruka38/daily/blob/master/ELYZA%2Bjson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

目的：
入力した文章を分類し
分類結果の出力をjson形式とする

In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.1/803.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline, TextStreamer, TextIteratorStreamer
import torch
from langchain import HuggingFacePipeline
from langchain.output_parsers import PydanticOutputParser #json
from langchain import PromptTemplate
from pydantic import BaseModel, Field, validator

In [3]:
model_name = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name)#, torch_dtype="auto")

#if torch.cuda.is_available():
#    model = model.to("cuda")

generation_config = GenerationConfig.from_pretrained(model_name)
generation_config.max_new_tokens = 256
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(model_id = model_name, pipeline=text_pipeline, model_kwargs={"temperature": 0})

tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [34]:
# 出力したいデータ構造を定義。
class InsuranceParam(BaseModel):
    label: str = Field(description="classification label")
    value: str = Field(description="あり/なし/増加/減少のどれか")

parser = PydanticOutputParser(pydantic_object=InsuranceParam)

In [35]:
template = """[INST] <<SYS>>
以下の例を基に入力文を分類してください。
出力は{format_instructions}に従い、labelが分類の名詞、valueは分類の値（あり/なし/増加/減少）とする。
入力文「介護が必要になる前に何か対策をしたい」
出力「介護:あり」

入力文「保険料を少なくしたい」
出力「保険料:減少」

入力文「保険の解約払戻金は欲しい」
出力「払戻金:あり」

入力文

<</SYS>>
{text}
[/INST]"""

elyza_prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [36]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"label": {"title": "Label", "description": "classification label", "type": "string"}, "value": {"title": "Value", "description": "\u3042\u308a/\u306a\u3057/\u5897\u52a0/\u6e1b\u5c11\u306e\u3069\u308c\u304b", "type": "string"}}, "required": ["label", "value"]}
```


In [37]:
#elyza_prompt.format(text="友達が介護の大変さを話してて自分も備えようと思った")

In [40]:
prompt_and_model = elyza_prompt | llm
text = "月々の保険料を抑えたい"

In [41]:
prompt_and_model.invoke({"text": text})

'  {\n    "label": "保険料",\n    "value": "減少"\n}'